# **Importations**

In [1]:
import pandas as pd
import gensim
from gensim.models import CoherenceModel
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
import string
from spacy.lang.en import English
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim_models
import itertools

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexandrequeant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alexandrequeant/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/alexandrequeant/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Il faut d'abord entrainer la classification sur la base du congrès, donc lancer la LDA dessus**

# **Traitement initial des journaux**

In [2]:
putback = ['prime', 'officials', 'security', 'news', 'working', 'games', 'jobs', 'campaign', 'services',
'civil', 'economic', 'information', 'political', 'election', 'court', 'office', 'vote', 'trump', 'control', 'job', 'price',
'donald trump', 'chinese', 'problems', 'concerns', 'minister', 'nation', 'policy', 'data', 'indian', 'congress',
'president', 'network', 'american', 'accused', 'government', 'money', 'investigation', 'facebook', 
'success', 'prices', 'twitter', 'book', 'politics',  'justice', 'claims', 'russia', 'law', 'technology',
'content', 'union', 'european', 'workers']

def construct_list_stopwords(list_putback_words=putback):
  '''
  Objectif:
  On importe la liste des stopwords :
  On a donc un dataframe des stopwords. On en tire une liste simple des stopwords, où l'on remet cependant des stopwords jugés significatifs, 
  comme expliqué pour le traitement des journaux :
  
  Arguments:
  list_putback_words -> list des mot jugés finalement signifcatifs 
  '''
  df_stopwords = pd.read_csv('/Users/alexandrequeant/Desktop/Statapp/sw1k.csv',
                names=['word', 'frequency', 'presence', 'doc_size_sum', 'type'],
                encoding='latin-1').drop(index=0)
  stopwords = df_stopwords['word'].unique()
  stopwords = list(set(stopwords) - set(list_putback_words))
  return stopwords


stop_words = construct_list_stopwords()


nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes(["tagger", "parser"])
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = set(stopwords.words('english'))

def extract_bigrams(n_grams):
    bigrams = []
    for i in range(len(n_grams)-1):
        bigram = f"{n_grams[i]} {n_grams[i+1]}"
        bigrams.append(bigram)
    return bigrams

def clean(text, list_stopwords, gram):
  '''
  Objectif:
  On va cleen le text
  On utilise la librairie SpaCy comme pour le traitement des journaux, et on enlève les nombres et la ponctuation avec la méthode translate. 
  
  Arguments:
  text -> str: le texte sur lequel on fait le processing
  list_stopwords -> list: mots à écarter !
  '''
  text = str(text).lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.translate(str.maketrans('', '', string.digits)) 
  # Tokenisation
  tokens = word_tokenize(text)
  # Enlever les caractères qui ne sont pas des lettres
  tokens = [re.sub('[^a-zA-Z]', '', token) for token in tokens]
  # Stemming
  stemmer = PorterStemmer()
  tokens_stemmed = [stemmer.stem(token) for token in tokens]
  filtered_words = [word for word in tokens_stemmed if not word.lower() in stop_words] #+ list(stop_words)
  # Transformation en bigramme
  if gram == 'bigram':
    filtered_words = extract_bigrams(filtered_words)
  return filtered_words

/Users/alexandrequeant/anaconda3/lib/python3.10/site-packages/spacy/language.py:1029: DeprecationWarning: [W096] The method `nlp.disable_pipes` is now deprecated - use `nlp.select_pipes` instead.
  warnings.warn(Warnings.W096, DeprecationWarning)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandrequeant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def treat_1(df, party):
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

def treat_2(df_1,df_2):
  df = pd.concat([df_1, df_2])
  df.dropna().reset_index(drop=True)
  return df

def treat_3(df):
  df['text'] = df['text'].apply(clean, args=(stop_words,'bigram'))
  return df 

In [4]:
df_Guardian = pd.read_csv('/Users/alexandrequeant/Desktop/Statapp/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/Users/alexandrequeant/Desktop/Statapp/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [5]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')
df_newspaper = treat_2(df_Guardian, df_DE)
df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
df_newspaper = df_newspaper.head(100)
df_newspaper = treat_3(df_newspaper)

/var/folders/6x/djr3ybxn3_d3j76smv6vqyfh0000gn/T/ipykernel_32887/640606626.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['party'] = 'Lab'
/var/folders/6x/djr3ybxn3_d3j76smv6vqyfh0000gn/T/ipykernel_32887/640606626.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=
/var/folders/6x/djr3ybxn3_d3j76smv6vqyfh0000gn/T/ipykernel_32887/640606626.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [6]:
df_newspaper

,Speaker,text,party
0,Danny Yadron,"[googl explor, explor strap, strap eye, eye de...",Lab
1,By Luke Edwards,"[googl chrome, chrome user, user window, windo...",Con
2,By Eleanore Robinson,"[twoyearold grace, grace race, race rescu, res...",Con
3,Stuart Dredge,"[appl ha, ha launch, launch endofyear, endofye...",Lab
4,By Aaron Brown,"[facebook ha, ha launch, launch app, app uk, u...",Con
...,...,...,...
95,Dominic Rushe,"[amazon doubl, doubl profit, profit bn, bn com...",Lab
96,Alex Hern,"[googl wa, wa remov, remov link, link copyrigh...",Lab
97,By Gary Jones,"[updat microsoft, microsoft ha, ha caus, caus ...",Con
98,By David Snelling,"[seem , creat, creat app, app appl, appl ha, ...",Con


In [29]:
#On crée notre document avec une liste de tous les most utilisés

doc = list(df_newspaper['text'])
doc = list(itertools.chain.from_iterable(doc))
doc = [word_tokenize(word) for word in doc]
doc = [extract_bigrams(couple) for couple in doc]

In [64]:
#On va créer notre dictionnaire en amont de la LDA

dictionary = gensim.corpora.Dictionary(doc)
dictionary.filter_extremes(no_below=2, no_above=0.1, keep_n=1000)
bow_corpus = [dictionary.doc2bow(word) for word in doc]

In [73]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 4, id2word = dictionary, passes = 500)

In [74]:
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

Topic: 0 -> Words: 0.026*"law enforc" + 0.025*"lumia lumia" + 0.020*"amazon prime" + 0.016*"amazon echo" + 0.015*"googl facebook" + 0.014*"pixel pixel" + 0.014*"googl pixel" + 0.012*"appl ha" + 0.010*"beat music" + 0.009*"platform ha"
Topic: 1 -> Words: 0.028*"pixel xl" + 0.015*"tech compani" + 0.012*"data centr" + 0.011*"googl mini" + 0.011*"crisi pregnanc" + 0.009*"echo devic" + 0.008*"thi includ" + 0.008*"compani ha" + 0.008*"torrent websit" + 0.008*"digit platform"
Topic: 2 -> Words: 0.025*"googl ha" + 0.018*"tech giant" + 0.015*"facebook ha" + 0.012*"facebook googl" + 0.012*"search engin" + 0.011*"iphon se" + 0.010*"facebook user" + 0.010*"amazon ha" + 0.010*"stream servic" + 0.009*"advertis revenu"
Topic: 3 -> Words: 0.023*"googl map" + 0.013*"santa tracker" + 0.012*"googl chrome" + 0.011*"thi wa" + 0.010*"googl earth" + 0.009*"pixel user" + 0.008*"news content" + 0.008*"strang planet" + 0.007*"thi featur" + 0.007*"wa shown"


In [75]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.7374975236038889


In [69]:
"""all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)"""

In [70]:
"""df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})"""

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.022, googl map)","(0.020, amazon prime)","(0.015, tech compani)","(0.012, facebook googl)","(0.012, appl ha)","(0.011, iphon se)","(0.010, amazon ha)","(0.010, facebook user)","(0.010, thi wa)","(0.009, pixel user)"
Topic 2,"(0.026, googl ha)","(0.018, tech giant)","(0.015, googl facebook)","(0.014, pixel pixel)","(0.014, googl pixel)","(0.012, googl chrome)","(0.012, data centr)","(0.012, search engin)","(0.011, googl mini)","(0.010, stream servic)"
Topic 3,"(0.016, facebook ha)","(0.016, amazon echo)","(0.011, crisi pregnanc)","(0.010, googl santa)","(0.009, ad revenu)","(0.009, iphon ipad)","(0.007, pixel devic)","(0.007, thi featur)","(0.007, web browser)","(0.007, bn revenu)"
Topic 4,"(0.028, pixel xl)","(0.027, law enforc)","(0.026, lumia lumia)","(0.013, santa tracker)","(0.010, beat music)","(0.010, googl earth)","(0.009, platform ha)","(0.008, torrent websit)","(0.008, microsoft ha)","(0.008, compani ha)"


In [76]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.184445 -0.139312       1        1  25.330002
1      0.010386  0.184697       2        1  24.962462
0     -0.200872 -0.116239       3        1  24.912664
3      0.006040  0.070854       4        1  24.794873, topic_info=               Term       Freq      Total Category  logprob  loglift
34         pixel xl  21.000000  21.000000  Default  30.0000  30.0000
28       law enforc  19.000000  19.000000  Default  29.0000  29.0000
37         googl ha  19.000000  19.000000  Default  28.0000  28.0000
983     lumia lumia  19.000000  19.000000  Default  27.0000  27.0000
969       googl map  17.000000  17.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
432    content youv   3.763000   4.349730   Topic4  -5.2829   1.2496
435      improv thi   3.763000   4.349730   Topic4  -5.2829   1.2496
436   includ advert   3.763000   4.349730   Topic4  -5.2829   1.2496
441  unsubscrib ani   3.763000   4.349730   Topic4  -5.2829   1.2496
828       news corp   3.749485   4.336215   Topic4  -5.2865   1.2491

[150 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
636       3  0.980728          ad revenu
437       3  0.936139          advert rd
956       1  0.914087    advertis revenu
265       3  0.970658        amazon echo
274       1  0.966579          amazon ha
...     ...       ...                ...
911       4  0.983877           wa shown
671       2  0.822342        web browser
916       2  0.908959        window user
954       1  0.931629  wolverin watchmen
433       1  0.941683       youv consent

[120 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])